In [ ]:
pip install hanja pandas openpyxl xlsxwriter

In [ ]:
# 호환 한자 → 표준 한자 변환
 
import pandas as pd
import unicodedata
import os

# 호환 한자 → 표준 한자 변환 함수
def normalize_text(x):
    if isinstance(x, str):
        return unicodedata.normalize("NFKC", x)
    return x

# 폴더 경로 설정
input_folder = r"경로"   # 변환할 엑셀 파일들이 있는 폴더
output_folder = os.path.join(input_folder, "normalized") # 결과 저장 폴더

os.makedirs(output_folder, exist_ok=True)

# 폴더 내 모든 엑셀 파일 처리
for filename in os.listdir(input_folder):
    if filename.lower().endswith(".xlsx"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f"normalized_{filename}")

        print(f"처리 중: {filename}")

        # 모든 시트 불러오기
        sheets = pd.read_excel(input_path, sheet_name=None)

        # 각 시트 정규화
        normalized_sheets = {}
        for sheet_name, df in sheets.items():
            for col in df.columns:
                df[col] = df[col].map(normalize_text)
            normalized_sheets[sheet_name] = df

        # 결과 저장
        with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
            for sheet_name, df in normalized_sheets.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ 모든 파일 변환 완료! 결과는 '{output_folder}'에 저장됩니다.")


In [ ]:
# 한자어_변환_사전 적용

import os
import re
import pandas as pd

# === 경로 설정 ===
DATA_DIR  = r"경로"   # 변환할 엑셀 파일들이 있는 폴더
DICT_PATH = os.path.join(DATA_DIR, "한자어_변환_사전.xlsx")
OUTPUT_DIR = os.path.join(DATA_DIR, "hanjadict")        # 결과 저장 폴더
os.makedirs(OUTPUT_DIR, exist_ok=True)                  # 폴더 없으면 생성
# =================

def build_space_insensitive_pattern(key: str) -> re.Pattern:
    key_no_space = re.sub(r"\s+", "", key)
    if not key_no_space:
        return None
    pattern_str = r"\s*".join(map(re.escape, list(key_no_space)))
    return re.compile(pattern_str)

def load_dictionary_patterns(dict_path: str):
    df = pd.read_excel(dict_path, dtype=str)

    if not {"Key","Value"}.issubset(set(df.columns)):
        raise ValueError("사전 파일에는 'Key', 'Value' 컬럼이 있어야 합니다.")

    df = df[["Key","Value"]].dropna()
    df["Key"]   = df["Key"].map(lambda s: s.strip())
    df["Value"] = df["Value"].map(lambda s: s.strip())

    df["_len_no_space"] = df["Key"].map(lambda s: len(re.sub(r"\s+","",s)))
    df = df[df["_len_no_space"] > 0].sort_values("_len_no_space", ascending=False)

    patterns = []
    for key, value in df[["Key","Value"]].itertuples(index=False, name=None):
        rx = build_space_insensitive_pattern(key)
        if rx is not None:
            patterns.append((rx, value))
    return patterns

def is_text_series(s: pd.Series) -> bool:
    if s.dtype == object:
        return True
    non_null = s.dropna()
    if len(non_null) == 0:
        return False
    str_ratio = (non_null.astype(str).apply(lambda x: isinstance(x, str))).mean()
    return str_ratio > 0.8

def make_replacer(patterns):
    def replace_cell(x):
        if pd.isna(x):
            return x
        text = str(x)
        for rx, val in patterns:
            text = rx.sub(val, text)
        return text
    return replace_cell

def process_excel_file(in_path, replacer):
    xls = pd.ExcelFile(in_path)
    base = os.path.splitext(os.path.basename(in_path))[0]
    out_path = os.path.join(OUTPUT_DIR, base + "_사전적용.xlsx")

    with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
        for sheet in xls.sheet_names:
            df = pd.read_excel(in_path, sheet_name=sheet, dtype=object)
            for col in df.columns:
                try:
                    if is_text_series(df[col]):
                        df[col] = df[col].map(replacer)
                except Exception:
                    pass
            df.to_excel(writer, sheet_name=sheet, index=False)
    return out_path

def main():
    patterns = load_dictionary_patterns(DICT_PATH)
    replacer = make_replacer(patterns)

    targets = []
    for fn in os.listdir(DATA_DIR):
        if fn.lower().endswith(".xlsx") and fn != os.path.basename(DICT_PATH):
            targets.append(os.path.join(DATA_DIR, fn))

    print(f"[INFO] 사전: {DICT_PATH}")
    print(f"[INFO] 변환 대상 파일 수: {len(targets)}")

    for p in targets:
        outp = process_excel_file(p, replacer)
        print(f" - 변환 완료: {os.path.basename(p)} → {os.path.basename(outp)}")

if __name__ == "__main__":
    main()


In [ ]:
# 한자 변환
# 한자-한글 데이터 출처: suminb님의 Python Library(https://github.com/suminb/hanja)

import os
import unicodedata
import pandas as pd
from typing import Iterable

# === 사용자 설정 ===
DATA_DIR   = r"경로"    # 엑셀 파일들이 있는 폴더
DICT_FILE  = "한자어_변환_사전.xlsx"                        # 변환에서 제외할 사전 파일명
OUT_SUFFIX = "_한자.xlsx"                                  # 출력 파일 접미사
OUTPUT_DIR = os.path.join(DATA_DIR, "hanja_converted")     # 결과 저장 하위 폴더
os.makedirs(OUTPUT_DIR, exist_ok=True)                     # 폴더 없으면 자동 생성
# ===================

# 1) hanja 라이브러리
#    pip install hanja
import hanja

def hanja_to_hangul(text: str) -> str:
    """호환한자 NFKC 정규화 후 한자→한글 변환"""
    if text is None:
        return text
    s = str(text)
    s = unicodedata.normalize("NFKC", s)   # 女 → 女, 罰課金 → 罰課金 등
    return hanja.translate(s, 'substitution')

def is_text_series(s: pd.Series) -> bool:
    """텍스트(문자열)로 볼만한 열만 변환"""
    if s.dtype == object:
        return True
    non_null = s.dropna()
    if len(non_null) == 0:
        return False
    try:
        str_ratio = (non_null.map(lambda x: isinstance(x, str))).mean()
    except Exception:
        str_ratio = 0.0
    return str_ratio > 0.8

def process_excel_file(in_path: str) -> str:
    """여러 시트 지원, 텍스트 열만 변환, OUTPUT_DIR에 저장"""
    xls = pd.ExcelFile(in_path)
    base_name = os.path.splitext(os.path.basename(in_path))[0]
    out_path = os.path.join(OUTPUT_DIR, base_name + OUT_SUFFIX)

    with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
        for sheet in xls.sheet_names:
            df = pd.read_excel(in_path, sheet_name=sheet)

            for col in df.columns:
                try:
                    if is_text_series(df[col]):
                        df[col] = df[col].map(lambda x: hanja_to_hangul(x) if pd.notna(x) else x)
                except Exception:
                    pass

            df.to_excel(writer, sheet_name=sheet, index=False)
    return out_path

def iter_target_excels(folder: str, exclude: Iterable[str]) -> Iterable[str]:
    for fn in os.listdir(folder):
        if not fn.lower().endswith(".xlsx"):
            continue
        if fn in exclude:
            continue
        yield os.path.join(folder, fn)

def main():
    exclude = {DICT_FILE}
    targets = list(iter_target_excels(DATA_DIR, exclude))
    print(f"[INFO] 변환 대상 파일 수: {len(targets)}")

    for p in targets:
        outp = process_excel_file(p)
        print(f" - 변환 완료: {os.path.basename(p)} → {os.path.basename(outp)}")

if __name__ == "__main__":
    main()
